# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

- ✅ **Streaming Inference**: Fetches live data from GEE (No massive downloads)
- ✅ **SwinIR Model**: Pretrained deep swin transformer for restoration
- ✅ **Hallucination Guardrails**: Ensures geospatial consistency

Enable GPU: `Runtime → Change runtime type → T4 GPU` (Recommended for speed)

In [ ]:
# 1. Clone repository & install
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo
%pip install -q -r requirements.txt
%pip install -q streamlit
!npm install -g localtunnel

In [ ]:
# 2. Check Device
import torch
if torch.cuda.is_available():
    print(f"✓ GPU Detected: {torch.cuda.get_device_name(0)}")
    DEVICE = 'cuda'
else:
    print("⚠ No GPU found. Using CPU.")
    DEVICE = 'cpu'

In [ ]:
# 3. GEE Authentication
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize(project='klymo-486313')
print("✓ GEE authenticated")

In [ ]:
# 4. Run Inference Demo
from src.models.swinir import load_swinir_model
from src.data.gee_fetcher import GEEFetcher
from src.inference.pipeline import SuperResolutionPipeline
from src.metrics.psnr import compute_psnr
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization

fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
fetcher.ee = ee
tile = fetcher.fetch_tile('delhi', tile_size=256)

pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(tile)
psnr = compute_psnr(results['bicubic'], results['sr'])

def zoom_crop(img):
    h, w = img.shape[:2]
    cy, cx = h//2, w//2
    return img[cy-150:cy+150, cx-150:cx+150]

fig, axes = plt.subplots(1, 2, figsize=(14, 7))
axes[0].imshow(to_8bit_visualization(zoom_crop(results['bicubic'])))
axes[0].set_title('Bicubic (Blurry)', fontsize=14)
axes[0].axis('off')
axes[1].imshow(to_8bit_visualization(zoom_crop(results['sr'])))
axes[1].set_title(f'SwinIR (Sharp) | PSNR: {psnr:.1f}dB', fontsize=14)
axes[1].axis('off')
plt.suptitle('Zoomed Center Crop', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Get Tunnel Password FIRST
import urllib.request
password = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
print("="*60)
print("🔑 TUNNEL PASSWORD (copy this now!):")
print(f"   {password}")
print("="*60)

In [ ]:
# 6. Launch Streamlit (click the .loca.lt URL, paste password from above)
!streamlit run app/streamlit_app.py &>/content/logs.txt & npx localtunnel --port 8501